#Methods - Scripts
\

#Step 1. Download the Epic Kitchens hand-object annotations files
\
  (The duration of this step depends on your internet connection as this requires the download of a very large ZIP file - 10Gb)

  1.1 - Use the url below to access the Epic Kitchens page hosting the file:
https://data.bris.ac.uk/data/dataset/3l8eci2oqgst92n14w2yqi5ytu
  
  1.2 - Click on the 'Complete download' button 

  1.3 - Unzip the file and extract all the folders to a directory of your choice

  1.4 - Access you gmail account. On the right top corner, you should see an icon with nine dots. Click on it and select Drive.

  1.5 - You should now be inside your Google Drive. Create a folder called 'hand-objects'. Inside this folder, create another folder called 'detections_root'. Enter the 'detections_root' folder and upload all the folders from the file you downloaded and extracted to your hardrive.

\
#Step 2. Setup analysis pipeline 



In [1]:
#Setting up goggle drive as main directory
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
#Downloading the dataset
!git clone https://github.com/epic-kitchens/epic-kitchens-100-hand-object-bboxes

Cloning into 'epic-kitchens-100-hand-object-bboxes'...
remote: Enumerating objects: 194, done.
remote: Counting objects: 100% (194/194), done.
remote: Compressing objects: 100% (123/123), done.
remote: Total 194 (delta 85), reused 154 (delta 54), pack-reused 0
Receiving objects: 100% (194/194), 946.33 KiB | 29.57 MiB/s, done.
Resolving deltas: 100% (85/85), done.


In [3]:
#Installing the epic-kitchens module
!pip install epic-kitchens

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 53 kB 2.8 MB/s 
     |████████████████████████████████| 51 kB 505 kB/s 
     |████████████████████████████████| 43 kB 2.1 MB/s 
     |████████████████████████████████| 43 kB 1.9 MB/s 
     |████████████████████████████████| 43 kB 1.5 MB/s 
ERROR: Cannot install epic-kitchens==1.1.1, epic-kitchens==1.1.2, epic-kitchens==1.2.0, epic-kitchens==1.2.1, epic-kitchens==1.3.0, epic-kitchens==1.4.0, epic-kitchens==1.5.0, epic-kitchens==1.6.0, epic-kitchens==1.6.2, epic-kitchens==1.6.3, epic-kitchens==1.7.0 and epic-kitchens==1.7.1 because these package versions have conflicting dependencies.

The conflict is caused by:
    epic-kitchens 1.7.1 depends on gulpio>=500
    epic-kitchens 1.7.0 depends on gulpio>=500
    epic-kitchens 1.6.3 depends on gulpio>=500
    epic-kitchens 1.6.2 depends on gulpio>=500
    epic-kitchens 1.6.0 depends on gulpio>=500
    epic

In [4]:
%cd /content/epic-kitchens-100-hand-object-bboxes

/content/epic-kitchens-100-hand-object-bboxes


In [5]:
#installing the automatic annotations 
!python3 setup.py install

running install
running bdist_egg
running egg_info
creating src/public_lib/epic_hand_object_detections.egg-info
writing src/public_lib/epic_hand_object_detections.egg-info/PKG-INFO
writing dependency_links to src/public_lib/epic_hand_object_detections.egg-info/dependency_links.txt
writing requirements to src/public_lib/epic_hand_object_detections.egg-info/requires.txt
writing top-level names to src/public_lib/epic_hand_object_detections.egg-info/top_level.txt
writing manifest file 'src/public_lib/epic_hand_object_detections.egg-info/SOURCES.txt'
writing manifest file 'src/public_lib/epic_hand_object_detections.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build
creating build/lib
creating build/lib/epic_kitchens
copying src/public_lib/epic_kitchens/__init__.py -> build/lib/epic_kitchens
creating build/lib/epic_kitchens/hoa
copying src/public_lib/epic_kitchens/hoa/io.py -> build/lib/epic_kitchens/hoa
copying s

In [6]:
%cd src/

/content/epic-kitchens-100-hand-object-bboxes/src


In [7]:
%cd public_lib/

/content/epic-kitchens-100-hand-object-bboxes/src/public_lib


In [8]:
#Importing the required libraries to load the automatic detections
from typing import Union, List
from pathlib import Path
import re
from ipywidgets import interact, IntSlider, Layout

from epic_kitchens.hoa import load_detections, DetectionRenderer
import PIL.Image

In [9]:
#Accessing the folder where all the frame annotations are
#glob turns all the content of the folders into a list 
#Counting the total number of annotation files

import glob

pkl_folder = sorted(glob.glob(r'/content/gdrive/MyDrive/hand-objects/detections_root/*/*'))

print("Total number of files: " + str(len(pkl_folder)))

Total number of files: 703


In [ ]:
#Reading one of the pkl files containing the annotations for participant P05, video 4
#Shows the information available in the original annotations so we could plan which info was relevant to answer our research question
video = str(pkl_folder[190])
video_id = video[57:]
detections_root = Path(video[0:56])
detections = load_detections(detections_root / (video_id))
detections

In [ ]:
#creating folder to save extracted data into csvs
import os

os.mkdir('/content/gdrive/MyDrive/HandVisualField_Paper/hands_csvs_4_analysis')

In [ ]:
%cd /content/gdrive/MyDrive/HandVisualField_Paper/hands_csvs_4_analysis

#Step 3. Determine total number of frames in the original dataset

In [ ]:
#'for' loop to extract total number of frames
detections_count = 0

for detec in pkl_folder:
  video_find = str(detec)
  detections_root = Path(video_find[0:56])
  video_id = video_find[57:]
  detections = load_detections(detections_root / (video_id))
  detections_count = detections_count + len(detections)

print(f"total_detections = {detections_count}")

total_detections = 20076005


#Step 4. Extract relevant data from original dataset into csv files
## Data extracted: Participant, frame number, hand side, hand image location in the image (x, y) and hand contact.

In [ ]:
#Extracts data from automatic annotations and saves it into a table format csv file

frame_height = 256 
frame_width = 456

handz = []
hands = []

for detec in pkl_folder:
  video_find = str(detec)
  detections_root = Path(video_find[0:56])
  video_id = video_find[57:]
  detections = load_detections(detections_root / (video_id))
  
  for detection in detections:
      detection.scale(height_factor=frame_height)
      fr = detection.frame_number
      for hand in detection.hands:
        hands.append(hand)
        hand_y = hand.bbox.center[1]
        hand_x = ((hand.bbox.left * frame_width) + ((hand.bbox.width * frame_width) / 2)) 
        hand_state = hand.state.name

        if hand.side.value == 0:
          z = video_id + "," + str(fr) + "," + "left" + "," + str(hand_y) + "," + str(round(hand_x, 2)) + "," + str(hand_state)
          handz.append(z)
        if hand.side.value == 1:
          z1 = video_id + "," + str(fr) + "," + "right" + "," + str(hand_y) + "," + str(round(hand_x, 2)) + "," + str(hand_state)
          handz.append(z1)
  x = "Participant, Frame, Hand, Centre_Y, Centre_X, Contact"
  handz.insert(0, x)

  textfile = open(video_id[:-4] + ".csv", "w")
  for element in handz:
    textfile.write(element + "\n")
  handz.clear()
  hands.clear()

#Step 5. Colating all csv files into a big dataset and manipulating the hand location coordenates so they follow a cartesian frame of reference. 
Note: computer image files have their coordenate origin (0,0) on the top left corner. This would introduce error when automating and plotting hand location in a cartesian frame of reference.

In [ ]:
#accessing the specific folder where the csv's are

import glob

csvs_folder = sorted(glob.glob(r'/content/gdrive/MyDrive/HandVisualField_Paper/hands_csvs_4_analysis/*.csv'))

print(len(csvs_folder))

703


In [ ]:
# Concatenate all data into one DataFrame

import pandas as pd

dfs = []

for filename in csvs_folder:

    dfs.append(pd.read_csv(filename))

big_frame = pd.concat(dfs, ignore_index=True)

big_frame[:10]

,Participant,Frame,Hand,Centre_Y,Centre_X,Contact
0,P01_01.pkl,211,right,147.5,321.0,STATIONARY_OBJECT
1,P01_01.pkl,212,right,141.5,324.5,STATIONARY_OBJECT
2,P01_01.pkl,213,right,126.0,328.5,PORTABLE_OBJECT
3,P01_01.pkl,214,right,122.0,329.0,PORTABLE_OBJECT
4,P01_01.pkl,215,right,118.5,326.5,PORTABLE_OBJECT
5,P01_01.pkl,216,right,118.5,326.5,PORTABLE_OBJECT
6,P01_01.pkl,217,right,118.0,322.5,PORTABLE_OBJECT
7,P01_01.pkl,218,right,117.0,322.5,PORTABLE_OBJECT
8,P01_01.pkl,219,right,123.0,317.0,PORTABLE_OBJECT
9,P01_01.pkl,220,right,124.0,317.0,PORTABLE_OBJECT


In [ ]:
#Turning Centre_Y column into a list
#Centre_Y was calculated based on image classification so 0 is the top left corner
#Centre_Y therefore needs to be converted to cartesian so 0 is the bottom left corner

Centre_Y_list = big_frame[" Centre_Y"].tolist()
Centre_Y_list[:10] #printing first 10 values to check they make sense

[147.5, 141.5, 126.0, 122.0, 118.5, 118.5, 118.0, 117.0, 123.0, 124.0]

In [ ]:
#Converting Y_centre to cartesian 
#Add values to new column
new_Centre_Y_list = []

for c in Centre_Y_list:
  n = 256 - c
  new_Centre_Y_list.append(n)

new_Centre_Y_list[:10]

[108.5, 114.5, 130.0, 134.0, 137.5, 137.5, 138.0, 139.0, 133.0, 132.0]

In [ ]:
#Adding new column to dataframe
big_frame.insert(loc = 3,
          column = 'cart_Centre_Y',
          value = new_Centre_Y_list)

# show the dataframe
big_frame

In [ ]:
#Removing Centre_Y column
big_frame.drop(big_frame.columns[4], axis=1, inplace=True)

In [ ]:
big_frame

In [ ]:
%cd /content/gdrive/MyDrive/HandVisualField_Paper

/content/gdrive/MyDrive/HandVisualField_Paper


In [ ]:
os.mkdir('/content/gdrive/MyDrive/HandVisualField_Paper/hands_big_Csv_4_analysis')

In [ ]:
%cd/content/gdrive/MyDrive/HandVisualField_Paper/hands_big_Csv_4_analysis

/content/gdrive/MyDrive/HandVisualField_Paper/hands_big_Csv_4_analysis


In [ ]:
#Saving the dataframe to a csv file
big_frame.to_csv(r'/content/gdrive/MyDrive/HandVisualField_Paper/hands_big_Csv_4_analysis/Final_Big_frame.csv', index = False)

In [10]:
import pandas as pd

new_df = pd.read_csv('/content/gdrive/MyDrive/HandVisualField_Paper/hands_big_Csv_4_analysis/Final_Big_frame.csv')
new_df

,Participant,Frame,Hand,cart_Centre_Y,Centre_X,Contact
0,P01_01.pkl,211,right,108.5,321.0,STATIONARY_OBJECT
1,P01_01.pkl,212,right,114.5,324.5,STATIONARY_OBJECT
2,P01_01.pkl,213,right,130.0,328.5,PORTABLE_OBJECT
3,P01_01.pkl,214,right,134.0,329.0,PORTABLE_OBJECT
4,P01_01.pkl,215,right,137.5,326.5,PORTABLE_OBJECT
...,...,...,...,...,...,...
31300196,P37_103.pkl,19700,right,53.0,399.5,NO_CONTACT
31300197,P37_103.pkl,19701,right,60.5,406.0,NO_CONTACT
31300198,P37_103.pkl,19702,right,69.5,414.0,NO_CONTACT
31300199,P37_103.pkl,19703,right,85.5,421.0,NO_CONTACT


In [13]:
hands_Self = new_df[new_df[' Contact'] == 'SELF_CONTACT']
hands_Self

,Participant,Frame,Hand,cart_Centre_Y,Centre_X,Contact
214,P01_01.pkl,479,left,35.5,149.0,SELF_CONTACT
1978,P01_01.pkl,2640,left,163.0,254.0,SELF_CONTACT
1980,P01_01.pkl,2641,left,163.5,253.5,SELF_CONTACT
2030,P01_01.pkl,2666,left,181.0,244.0,SELF_CONTACT
2037,P01_01.pkl,2670,right,214.0,282.5,SELF_CONTACT
...,...,...,...,...,...,...
31299344,P37_103.pkl,18837,right,45.0,331.5,SELF_CONTACT
31299347,P37_103.pkl,18839,left,40.0,306.0,SELF_CONTACT
31299348,P37_103.pkl,18839,right,47.5,334.0,SELF_CONTACT
31299349,P37_103.pkl,18840,left,32.0,308.5,SELF_CONTACT


In [14]:
hands_Self.Participant.unique()

array(['P01_01.pkl', 'P01_02.pkl', 'P01_03.pkl', 'P01_04.pkl',
       'P01_05.pkl', 'P01_06.pkl', 'P01_07.pkl', 'P01_08.pkl',
       'P01_09.pkl', 'P01_10.pkl', 'P01_101.pkl', 'P01_102.pkl',
       'P01_103.pkl', 'P01_104.pkl', 'P01_105.pkl', 'P01_106.pkl',
       'P01_107.pkl', 'P01_108.pkl', 'P01_109.pkl', 'P01_11.pkl',
       'P01_12.pkl', 'P01_13.pkl', 'P01_14.pkl', 'P01_15.pkl',
       'P01_16.pkl', 'P01_17.pkl', 'P01_18.pkl', 'P01_19.pkl',
       'P02_01.pkl', 'P02_02.pkl', 'P02_03.pkl', 'P02_04.pkl',
       'P02_06.pkl', 'P02_09.pkl', 'P02_101.pkl', 'P02_102.pkl',
       'P02_103.pkl', 'P02_104.pkl', 'P02_105.pkl', 'P02_106.pkl',
       'P02_107.pkl', 'P02_108.pkl', 'P02_109.pkl', 'P02_110.pkl',
       'P02_111.pkl', 'P02_112.pkl', 'P02_113.pkl', 'P02_114.pkl',
       'P02_115.pkl', 'P02_116.pkl', 'P02_117.pkl', 'P02_118.pkl',
       'P02_119.pkl', 'P02_12.pkl', 'P02_120.pkl', 'P02_121.pkl',
       'P02_122.pkl', 'P02_123.pkl', 'P02_124.pkl', 'P02_125.pkl',
       'P02_126.pkl',

#Step 6. Determining the number of processed frames containing hands (i.e. excluding all duplicated frames)  

In [ ]:
double_df = len(new_df[new_df[' Frame'].duplicated()])
double_df

31077790

In [ ]:
single_df = len(new_df) - double_df
single_df

222411

In [ ]:
half_doubles = double_df / 2
half_doubles

15538895.0

In [ ]:
real_hand_df = half_doubles + single_df
real_hand_df

15761306.0

In [ ]:
p = (real_hand_df / detections_count) * 100
print('Proportion of frames with hands in visual field = ' + str(p))

Proportion of frames with hands in visual field = 78.50817929164691


#Step 7. Extracting data according to hand location by vertical hemi fields, horizontal hemifields and quadrants 
## 7.1 - Overall sample


In [ ]:
#for loop to pull data from the csv and group info into the columns mentioned below - hand position in space

df = new_df

hands_csv = []

vid_id = str(df['Participant'])
vid_name = vid_id[:-4]
hands_left = df[' Hand'] == 'left'
hands_right = df[' Hand'] == 'right'
hands_lower = df['cart_Centre_Y'] < 128
hands_upper = df['cart_Centre_Y'] >= 128
hands_Semi_Right = df[' Centre_X'] > 228
hands_Semi_Left = df[' Centre_X'] <= 228

values = str(len(df[' Hand'])) + "," + str(len(df[hands_left])) + "," + str(len(df[hands_left & hands_upper])) + "," + str(len(df[hands_left & hands_lower])) + "," + str(len(df[hands_left & hands_Semi_Left])) + "," + str(len(df[hands_left & hands_Semi_Right])) + "," + str(len(df[hands_left & hands_upper & hands_Semi_Left])) + "," + str(len(df[hands_left & hands_lower & hands_Semi_Left])) + "," + str(len(df[hands_left & hands_upper & hands_Semi_Right])) + "," + str(len(df[hands_left & hands_lower & hands_Semi_Right])) + "," + str(len(df[hands_right])) + "," + str(len(df[hands_right & hands_upper])) + "," + str(len(df[hands_right & hands_lower]))+ ","  + str(len(df[hands_right & hands_Semi_Left])) + "," +str(len(df[hands_right & hands_Semi_Right])) + "," + str(len(df[hands_right & hands_upper & hands_Semi_Left])) + "," + str(len(df[hands_right & hands_lower & hands_Semi_Left])) + "," + str(len(df[hands_right & hands_upper & hands_Semi_Right])) + "," + str(len(df[hands_right & hands_lower & hands_Semi_Right])) 
  
hands_csv.append(values)
                                 
x = "hands_total, hands_left, left_upper, left_lower, left_ipsi, left_contra, left_ULquad, left_LLquad, leftURquad, leftLRquad, hands_right, right_upper, right_lower, right_contra, right_ipsi, right_ULquad, right_LLquad, right_URquad, right_LRquad" 
  
hands_csv.insert(0, x)

textfile = open("hands_dataset_analysis.csv", "w")
for element in hands_csv:
    textfile.write(element + "\n")
textfile.close()

In [ ]:
crushed_df = pd.read_csv('/content/gdrive/MyDrive/HandVisualField_Paper/hands_big_Csv_4_analysis/hands_dataset_analysis.csv')
crushed_df

,hands_total,hands_left,left_upper,left_lower,left_ipsi,left_contra,left_ULquad,left_LLquad,leftURquad,leftLRquad,hands_right,right_upper,right_lower,right_contra,right_ipsi,right_ULquad,right_LLquad,right_URquad,right_LRquad
0,31300201,15176277,3250461,11925816,12786258,2390019,2781452,10004806,469009,1921010,16123924,4277873,11846051,1487604,14636320,338845,1148759,3939028,10697292


##7.1.1 - Extracting data according to object interaction

In [ ]:
%cd /content/gdrive/MyDrive/HandVisualField_Paper/hands_big_Csv_4_analysis

/content/gdrive/MyDrive/HandVisualField_Paper/hands_big_Csv_4_analysis


In [ ]:
#for loop to pull data from overall csv and group info into visual field areas and no object interaction
df_NC = new_df

noContact_csv = []

vid_id = str(df_NC['Participant'])
vid_name = vid_id[:-4]
hands_left = df_NC[' Hand'] == 'left'
hands_right = df_NC[' Hand'] == 'right'
hands_lower = df_NC['cart_Centre_Y'] < 128
hands_upper = df_NC['cart_Centre_Y'] >= 128
hands_Semi_Right = df_NC[' Centre_X'] > 228
hands_Semi_Left = df_NC[' Centre_X'] <= 228
hands_noContact = df_NC[' Contact'] == 'NO_CONTACT'
hands_Portable = df_NC[' Contact'] == 'PORTABLE_OBJECT'
hands_Stationary  = df_NC[' Contact'] == 'STATIONARY_OBJECT'
hands_Self = df_NC[' Contact'] == 'SELF_CONTACT'
  
values_NC = str(len(df_NC[' Hand'])) + "," + str(len(df_NC[hands_noContact])) + "," + str(len(df_NC[hands_noContact & hands_left])) + "," + str(len(df_NC[hands_noContact & hands_left & hands_upper])) + "," + str(len(df_NC[hands_noContact & hands_left & hands_lower])) + "," + str(len(df_NC[hands_noContact & hands_left & hands_Semi_Left])) + "," + str(len(df_NC[hands_noContact & hands_left & hands_Semi_Right])) + "," + str(len(df_NC[hands_noContact & hands_left & hands_upper & hands_Semi_Left])) + "," + str(len(df_NC[hands_noContact & hands_left & hands_lower & hands_Semi_Left])) + "," + str(len(df_NC[hands_noContact & hands_left & hands_upper & hands_Semi_Right])) + "," + str(len(df_NC[hands_noContact & hands_left & hands_lower & hands_Semi_Right])) + "," + str(len(df_NC[hands_noContact & hands_right])) + "," + str(len(df_NC[hands_noContact & hands_right & hands_upper])) + "," + str(len(df_NC[hands_noContact & hands_right & hands_lower])) + "," + str(len(df_NC[hands_noContact & hands_right & hands_Semi_Left])) + "," + str(len(df_NC[hands_noContact & hands_right & hands_Semi_Right])) + "," + str(len(df_NC[hands_noContact & hands_right & hands_upper & hands_Semi_Left])) + "," + str(len(df_NC[hands_noContact & hands_right & hands_lower & hands_Semi_Left])) + "," + str(len(df_NC[hands_noContact & hands_right & hands_upper & hands_Semi_Right])) + "," + str(len(df_NC[hands_noContact & hands_right & hands_lower & hands_Semi_Right])) 
noContact_csv.append(values_NC)
                                      
x = "hands_total, noContact_total, noContact_left, noContact_left_upper, noContact_left_lower, noContact_left_ipsi, noContact_left_contra, noContact_left_ULquad, noContact_left_LLquad, noContact_left_URquad, noContact_left_LRquad, noContact_hands_right, noContact_right_upper, noContact_right_lower, noContact_right_contra, noContact_right_ipsi, noContact_right_ULquad, noContact_right_LLquad, noContact_right_URquad, noContact_right_LRquad"
noContact_csv.insert(0, x)

textfile_NC = open("hands_noContact_analysis.csv", "w")
for element in noContact_csv:
    textfile_NC.write(element + "\n")
textfile_NC.close()

In [ ]:
df_NC_crushed = pd.read_csv('/content/gdrive/MyDrive/HandVisualField_Paper/hands_big_Csv_4_analysis/hands_noContact_analysis.csv')
df_NC_crushed

,hands_total,noContact_total,noContact_left,noContact_left_upper,noContact_left_lower,noContact_left_ipsi,noContact_left_contra,noContact_left_ULquad,noContact_left_LLquad,noContact_left_URquad,noContact_left_LRquad,noContact_hands_right,noContact_right_upper,noContact_right_lower,noContact_right_contra,noContact_right_ipsi,noContact_right_ULquad,noContact_right_LLquad,noContact_right_URquad,noContact_right_LRquad
0,31300201,3968863,2091966,287840,1804126,1751737,340229,241528,1510209,46312,293917,1876897,388153,1488744,241505,1635392,44811,196694,343342,1292050


In [ ]:
#for loop to pull data from overall csv and group info into visual field areas and portable object interaction

import pandas as pd
df = new_df

hands_csv2 = []

vid_id = str(df['Participant'])
vid_name = vid_id[:-4]
hands_left = df[' Hand'] == 'left'
hands_right = df[' Hand'] == 'right'
hands_lower = df['cart_Centre_Y'] < 128
hands_upper = df['cart_Centre_Y'] >= 128
hands_Semi_Right = df[' Centre_X'] > 228
hands_Semi_Left = df[' Centre_X'] <= 228
hands_noContact = df[' Contact'] == 'NO_CONTACT'
hands_Portable = df[' Contact'] == 'PORTABLE_OBJECT'
hands_Stationary  = df[' Contact'] == 'STATIONARY_OBJECT'
hands_Self = df[' Contact'] == 'SELF_CONTACT'
  
values1 = str(len(df[' Hand'])) + "," + str(len(df[hands_Portable])) + "," + str(len(df[hands_Portable & hands_left])) + "," + str(len(df[hands_Portable & hands_left & hands_upper])) + "," + str(len(df[hands_Portable & hands_left & hands_lower])) + "," + str(len(df[hands_Portable & hands_left & hands_Semi_Left])) + "," + str(len(df[hands_Portable & hands_left & hands_Semi_Right])) + "," + str(len(df[hands_Portable & hands_left & hands_upper & hands_Semi_Left])) + "," + str(len(df[hands_Portable & hands_left & hands_lower & hands_Semi_Left])) + "," + str(len(df[hands_Portable & hands_left & hands_upper & hands_Semi_Right])) + "," + str(len(df[hands_Portable & hands_left & hands_lower & hands_Semi_Right])) + "," + str(len(df[hands_Portable & hands_right])) + "," + str(len(df[hands_Portable & hands_right & hands_upper])) + "," + str(len(df[hands_Portable & hands_right & hands_lower])) + "," + str(len(df[hands_Portable & hands_right & hands_Semi_Left])) + "," + str(len(df[hands_Portable & hands_right & hands_Semi_Right])) + "," + str(len(df[hands_Portable & hands_right & hands_upper & hands_Semi_Left])) + "," + str(len(df[hands_Portable & hands_right & hands_lower & hands_Semi_Left])) + "," + str(len(df[hands_Portable & hands_right & hands_upper & hands_Semi_Right])) + "," + str(len(df[hands_Portable & hands_right & hands_lower & hands_Semi_Right])) 
hands_csv2.append(values1)
                                      
x = "hands_total, portable_total, portable_left, portable_left_upper, portable_left_lower, portable_left_ipsi, portable_left_contra, portable_left_ULquad, portable_left_LLquad, portable_left_URquad, portable_left_LRquad, portable_hands_right, portablet_right_upper, portable_right_lower, portable_right_contra, portable_right_ipsi, portable_right_ULquad, portable_right_LLquad, portable_right_URquad, portable_right_LRquad"
hands_csv2.insert(0, x)

textfile = open("obj_interaction_Portable.csv", "w")
for element in hands_csv2:
    textfile.write(element + "\n")
textfile.close()

In [ ]:
port_df = pd.read_csv('/content/gdrive/MyDrive/HandVisualField_Paper/hands_big_Csv_4_analysis/obj_interaction_Portable.csv')
port_df

,hands_total,portable_total,portable_left,portable_left_upper,portable_left_lower,portable_left_ipsi,portable_left_contra,portable_left_ULquad,portable_left_LLquad,portable_left_URquad,portable_left_LRquad,portable_hands_right,portablet_right_upper,portable_right_lower,portable_right_contra,portable_right_ipsi,portable_right_ULquad,portable_right_LLquad,portable_right_URquad,portable_right_LRquad
0,31300201,26054826,12463499,2818554,9644945,10551591,1911908,2422196,8129395,396358,1515550,13591327,3698215,9893112,1143240,12448087,267460,875780,3430755,9017332


In [ ]:
#for loop to pull data from overall csv and group info into visual field areas and stationary object interaction

import pandas as pd
df = new_df

hands_csv3 = []

vid_id = str(df['Participant'])
vid_name = vid_id[:-4]
hands_left = df[' Hand'] == 'left'
hands_right = df[' Hand'] == 'right'
hands_lower = df['cart_Centre_Y'] < 128
hands_upper = df['cart_Centre_Y'] >= 128
hands_Semi_Right = df[' Centre_X'] > 228
hands_Semi_Left = df[' Centre_X'] <= 228
hands_noContact = df[' Contact'] == 'NO_CONTACT'
hands_Portable = df[' Contact'] == 'PORTABLE_OBJECT'
hands_Stationary  = df[' Contact'] == 'STATIONARY_OBJECT'
hands_Self = df[' Contact'] == 'SELF_CONTACT'
  
values1 = str(len(df[' Hand'])) + "," + str(len(df[hands_Stationary])) + "," + str(len(df[hands_Stationary & hands_left])) + "," + str(len(df[hands_Stationary & hands_left & hands_upper])) + "," + str(len(df[hands_Stationary & hands_left & hands_lower])) + "," + str(len(df[hands_Stationary & hands_left & hands_Semi_Left])) + "," + str(len(df[hands_Stationary & hands_left & hands_Semi_Right])) + "," + str(len(df[hands_Stationary & hands_left & hands_upper & hands_Semi_Left])) + "," + str(len(df[hands_Stationary & hands_left & hands_lower & hands_Semi_Left])) + "," + str(len(df[hands_Stationary & hands_left & hands_upper & hands_Semi_Right])) + "," + str(len(df[hands_Stationary & hands_left & hands_lower & hands_Semi_Right])) + "," + str(len(df[hands_Stationary & hands_right])) + "," + str(len(df[hands_Stationary & hands_right & hands_upper])) + "," + str(len(df[hands_Stationary & hands_right & hands_lower])) + "," + str(len(df[hands_Stationary & hands_right & hands_Semi_Left])) + "," + str(len(df[hands_Stationary & hands_right & hands_Semi_Right])) + "," + str(len(df[hands_Stationary & hands_right & hands_upper & hands_Semi_Left])) + "," + str(len(df[hands_Stationary & hands_right & hands_lower & hands_Semi_Left])) + "," + str(len(df[hands_Stationary & hands_right & hands_upper & hands_Semi_Right])) + "," + str(len(df[hands_Stationary & hands_right & hands_lower & hands_Semi_Right])) 
hands_csv3.append(values1)
                                      
x = "hands_total, Stationary_total, Stationary_left, Stationary_left_upper, Stationary_left_lower, Stationary_left_ipsi, Stationary_left_contra, Stationary_left_ULquad, Stationary_left_LLquad, Stationary_left_URquad, Stationary_left_LRquad, Stationary_hands_right, Stationary_right_upper, Stationary_right_lower, Stationary_right_contra, Stationary_right_ipsi, Stationary_right_ULquad, Stationary_right_LLquad, Stationary_right_URquad, Stationary_right_LRquad"
hands_csv3.insert(0, x)

textfile = open("obj_interaction_Stationary.csv", "w")
for element in hands_csv3:
    textfile.write(element + "\n")
textfile.close()

In [ ]:
stat_df = pd.read_csv('/content/gdrive/MyDrive/HandVisualField_Paper/hands_big_Csv_4_analysis/obj_interaction_Stationary.csv')
stat_df

,hands_total,Stationary_total,Stationary_left,Stationary_left_upper,Stationary_left_lower,Stationary_left_ipsi,Stationary_left_contra,Stationary_left_ULquad,Stationary_left_LLquad,Stationary_left_URquad,Stationary_left_LRquad,Stationary_hands_right,Stationary_right_upper,Stationary_right_lower,Stationary_right_contra,Stationary_right_ipsi,Stationary_right_ULquad,Stationary_right_LLquad,Stationary_right_URquad,Stationary_right_LRquad
0,31300201,901915,427683,106388,321295,368849,58834,92807,276042,13581,45253,474232,156070,318162,52525,421707,16284,36241,139786,281921


In [ ]:
#for loop to pull data from overall csv and group info into visual field areas and contra-lateral hand interaction

import pandas as pd
df = new_df

hands_csv4 = []

vid_id = str(df['Participant'])
vid_name = vid_id[:-4]
hands_left = df[' Hand'] == 'left'
hands_right = df[' Hand'] == 'right'
hands_lower = df['cart_Centre_Y'] < 128
hands_upper = df['cart_Centre_Y'] >= 128
hands_Semi_Right = df[' Centre_X'] > 228
hands_Semi_Left = df[' Centre_X'] <= 228
hands_noContact = df[' Contact'] == 'NO_CONTACT'
hands_Portable = df[' Contact'] == 'PORTABLE_OBJECT'
hands_Stationary  = df[' Contact'] == 'STATIONARY_OBJECT'
hands_Self = df[' Contact'] == 'SELF_CONTACT'
  
values1 = str(len(df[' Hand'])) + "," + str(len(df[hands_Self])) + "," + str(len(df[hands_Self & hands_left])) + "," + str(len(df[hands_Self & hands_left & hands_upper])) + "," + str(len(df[hands_Self & hands_left & hands_lower])) + "," + str(len(df[hands_Self & hands_left & hands_Semi_Left])) + "," + str(len(df[hands_Self & hands_left & hands_Semi_Right])) + "," + str(len(df[hands_Self & hands_left & hands_upper & hands_Semi_Left])) + "," + str(len(df[hands_Self & hands_left & hands_lower & hands_Semi_Left])) + "," + str(len(df[hands_Self & hands_left & hands_upper & hands_Semi_Right])) + "," + str(len(df[hands_Self & hands_left & hands_lower & hands_Semi_Right])) + "," + str(len(df[hands_Self & hands_right])) + "," + str(len(df[hands_Self & hands_right & hands_upper])) + "," + str(len(df[hands_Self & hands_right & hands_lower])) + "," + str(len(df[hands_Self & hands_right & hands_Semi_Left])) + "," + str(len(df[hands_Self & hands_right & hands_Semi_Right])) + "," + str(len(df[hands_Self & hands_right & hands_upper & hands_Semi_Left])) + "," + str(len(df[hands_Self & hands_right & hands_lower & hands_Semi_Left])) + "," + str(len(df[hands_Self & hands_right & hands_upper & hands_Semi_Right])) + "," + str(len(df[hands_Self & hands_right & hands_lower & hands_Semi_Right])) 
hands_csv4.append(values1)
                                      
x = "hands_total, hands_Self_total, hands_Self_left, hands_Self_left_upper, hands_Self_left_lower, hands_Self_left_ipsi, hands_Self_left_contra, hands_Self_left_ULquad, hands_Self_left_LLquad, hands_Self_left_URquad, hands_Self_left_LRquad, hands_Self_hands_right, hands_Self_right_upper, hands_Self_right_lower, hands_Self_right_contra, hands_Self_right_ipsi, hands_Self_right_ULquad, hands_Self_right_LLquad, hands_Self_right_URquad, hands_Self_right_LRquad"
hands_csv4.insert(0, x)

textfile = open("obj_interaction_self.csv", "w")
for element in hands_csv4:
    textfile.write(element + "\n")
textfile.close()

In [ ]:
self_df = pd.read_csv('/content/gdrive/MyDrive/HandVisualField_Paper/hands_big_Csv_4_analysis/obj_interaction_self.csv')
self_df

,hands_total,hands_Self_total,hands_Self_left,hands_Self_left_upper,hands_Self_left_lower,hands_Self_left_ipsi,hands_Self_left_contra,hands_Self_left_ULquad,hands_Self_left_LLquad,hands_Self_left_URquad,hands_Self_left_LRquad,hands_Self_hands_right,hands_Self_right_upper,hands_Self_right_lower,hands_Self_right_contra,hands_Self_right_ipsi,hands_Self_right_ULquad,hands_Self_right_LLquad,hands_Self_right_URquad,hands_Self_right_LRquad
0,31300201,374597,193129,37679,155450,114081,79048,24921,89160,12758,66290,181468,35435,146033,50334,131134,10290,40044,25145,105989


#Extracting data from dataset by hand dominance
## 7.2 - Left handed participants only


In [ ]:
#accessing the specific folder where the csv's are

import glob

csvs_folder_left = sorted(glob.glob(r'/content/gdrive/MyDrive/HandVisualField_Paper/hands_csvs_left_4_analysis/*.csv'))

print(len(csvs_folder_left))

138


In [ ]:
# Concatenate all data into one DataFrame

import pandas as pd

dfs_left = []

for filename in csvs_folder_left:

    dfs_left.append(pd.read_csv(filename))

big_frame_left = pd.concat(dfs_left, ignore_index=True)

big_frame_left

In [ ]:
#Turning Centre_Y column into a list
#Centre_Y was calculated based on image classification so 0 is the top left corner
#Centre_Y therefore needs to be converted to cartesian so 0 is the bottom left corner

Centre_Y_list = big_frame_left[" Centre_Y"].tolist()
Centre_Y_list[:10] #printing first 10 values to check they make sense

[54.0, 55.0, 55.5, 55.0, 207.5, 203.5, 218.0, 216.0, 224.5, 226.5]

In [ ]:
#Converting Y_centre to cartesian 
#Add values to new column
new_Centre_Y_list = []

for c in Centre_Y_list:
  n = 256 - c
  new_Centre_Y_list.append(n)

new_Centre_Y_list[:10]

[202.0, 201.0, 200.5, 201.0, 48.5, 52.5, 38.0, 40.0, 31.5, 29.5]

In [ ]:
#Adding new column to dataframe
big_frame_left.insert(loc = 3,
          column = 'cart_Centre_Y',
          value = new_Centre_Y_list)

# show the dataframe
big_frame_left

In [ ]:
#Removing Centre_Y column
big_frame_left.drop(big_frame_left.columns[4], axis=1, inplace=True)

In [ ]:
big_frame_left

In [ ]:
#Saving the dataframe to a csv file
big_frame_left.to_csv(r'/content/gdrive/MyDrive/HandVisualField_Paper/hands_big_Csv_4_analysis_test/Final_Big_frame_Left.csv', index = False)

In [ ]:
#for loop to pull data from the csv and group info into the columns mentioned below - hand position in space in left dominant participants

df = pd.read_csv('/content/gdrive/MyDrive/HandVisualField_Paper/hands_big_Csv_4_analysis/Final_Big_frame_Left.csv')

hands_csv = []

vid_id = str(df['Participant'])
vid_name = vid_id[:-4]
hands_left = df[' Hand'] == 'left'
hands_right = df[' Hand'] == 'right'
hands_lower = df['cart_Centre_Y'] < 128
hands_upper = df['cart_Centre_Y'] >= 128
hands_Semi_Right = df[' Centre_X'] > 228
hands_Semi_Left = df[' Centre_X'] <= 228

values = str(len(df[' Hand'])) + "," + str(len(df[hands_left])) + "," + str(len(df[hands_left & hands_upper])) + "," + str(len(df[hands_left & hands_lower])) + "," + str(len(df[hands_left & hands_Semi_Left])) + "," + str(len(df[hands_left & hands_Semi_Right])) + "," + str(len(df[hands_left & hands_upper & hands_Semi_Left])) + "," + str(len(df[hands_left & hands_lower & hands_Semi_Left])) + "," + str(len(df[hands_left & hands_upper & hands_Semi_Right])) + "," + str(len(df[hands_left & hands_lower & hands_Semi_Right])) + "," + str(len(df[hands_right])) + "," + str(len(df[hands_right & hands_upper])) + "," + str(len(df[hands_right & hands_lower]))+ ","  + str(len(df[hands_right & hands_Semi_Left])) + "," +str(len(df[hands_right & hands_Semi_Right])) + "," + str(len(df[hands_right & hands_upper & hands_Semi_Left])) + "," + str(len(df[hands_right & hands_lower & hands_Semi_Left])) + "," + str(len(df[hands_right & hands_upper & hands_Semi_Right])) + "," + str(len(df[hands_right & hands_lower & hands_Semi_Right])) 
  
hands_csv.append(values)
                                 
x = "hands_total, hands_left, left_upper, left_lower, left_ipsi, left_contra, left_ULquad, left_LLquad, leftURquad, leftLRquad, hands_right, right_upper, right_lower, right_contra, right_ipsi, right_ULquad, right_LLquad, right_URquad, right_LRquad" 
  
hands_csv.insert(0, x)

textfile = open("hands_dataset_analysis_Left.csv", "w")
for element in hands_csv:
    textfile.write(element + "\n")
textfile.close()

In [ ]:
crushed_df_Left = pd.read_csv('/content/gdrive/MyDrive/HandVisualField_Paper/hands_big_Csv_4_analysis/hands_dataset_analysis_Left.csv')
crushed_df_Left

,hands_total,hands_left,left_upper,left_lower,left_ipsi,left_contra,left_ULquad,left_LLquad,leftURquad,leftLRquad,hands_right,right_upper,right_lower,right_contra,right_ipsi,right_ULquad,right_LLquad,right_URquad,right_LRquad
0,4333315,2217155,499638,1717517,1867318,349837,458797,1408521,40841,308996,2116160,615043,1501117,193150,1923010,68470,124680,546573,1376437


In [ ]:
import pandas as pd
ldf = pd.read_csv('/content/gdrive/MyDrive/HandVisualField_Paper/hands_big_Csv_4_analysis/Final_Big_frame_Left.csv')
ldf

,Participant,Frame,Hand,cart_Centre_Y,Centre_X,Contact
0,P12_01.pkl,1,left,202.0,400.0,NO_CONTACT
1,P12_01.pkl,2,left,201.0,400.5,NO_CONTACT
2,P12_01.pkl,3,left,200.5,398.5,NO_CONTACT
3,P12_01.pkl,4,left,201.0,399.0,NO_CONTACT
4,P12_01.pkl,7,right,48.5,421.5,NO_CONTACT
...,...,...,...,...,...,...
4333310,P32_10.pkl,24272,left,25.5,221.5,NO_CONTACT
4333311,P32_10.pkl,24307,right,21.0,233.5,NO_CONTACT
4333312,P32_10.pkl,24308,right,21.5,234.5,NO_CONTACT
4333313,P32_10.pkl,24311,right,14.5,221.0,NO_CONTACT


In [ ]:
%cd /content/gdrive/MyDrive/HandVisualField_Paper/hands_big_Csv_4_analysis

/content/gdrive/MyDrive/HandVisualField_Paper/hands_big_Csv_4_analysis


In [ ]:
#for loop to pull data from all csv's and group info into the columns mentioned below - object interaction-noContact
df_NC = ldf

noContact_csv = []

vid_id = str(df_NC['Participant'])
vid_name = vid_id[:-4]
hands_left = df_NC[' Hand'] == 'left'
hands_right = df_NC[' Hand'] == 'right'
hands_lower = df_NC['cart_Centre_Y'] < 128
hands_upper = df_NC['cart_Centre_Y'] >= 128
hands_Semi_Right = df_NC[' Centre_X'] > 228
hands_Semi_Left = df_NC[' Centre_X'] <= 228
hands_noContact = df_NC[' Contact'] == 'NO_CONTACT'
hands_Portable = df_NC[' Contact'] == 'PORTABLE_OBJECT'
hands_Stationary  = df_NC[' Contact'] == 'STATIONARY_OBJECT'
hands_Self = df_NC[' Contact'] == 'SELF_CONTACT'
  
values_NC = str(len(df_NC[' Hand'])) + "," + str(len(df_NC[hands_noContact])) + "," + str(len(df_NC[hands_noContact & hands_left])) + "," + str(len(df_NC[hands_noContact & hands_left & hands_upper])) + "," + str(len(df_NC[hands_noContact & hands_left & hands_lower])) + "," + str(len(df_NC[hands_noContact & hands_left & hands_Semi_Left])) + "," + str(len(df_NC[hands_noContact & hands_left & hands_Semi_Right])) + "," + str(len(df_NC[hands_noContact & hands_left & hands_upper & hands_Semi_Left])) + "," + str(len(df_NC[hands_noContact & hands_left & hands_lower & hands_Semi_Left])) + "," + str(len(df_NC[hands_noContact & hands_left & hands_upper & hands_Semi_Right])) + "," + str(len(df_NC[hands_noContact & hands_left & hands_lower & hands_Semi_Right])) + "," + str(len(df_NC[hands_noContact & hands_right])) + "," + str(len(df_NC[hands_noContact & hands_right & hands_upper])) + "," + str(len(df_NC[hands_noContact & hands_right & hands_lower])) + "," + str(len(df_NC[hands_noContact & hands_right & hands_Semi_Left])) + "," + str(len(df_NC[hands_noContact & hands_right & hands_Semi_Right])) + "," + str(len(df_NC[hands_noContact & hands_right & hands_upper & hands_Semi_Left])) + "," + str(len(df_NC[hands_noContact & hands_right & hands_lower & hands_Semi_Left])) + "," + str(len(df_NC[hands_noContact & hands_right & hands_upper & hands_Semi_Right])) + "," + str(len(df_NC[hands_noContact & hands_right & hands_lower & hands_Semi_Right])) 
noContact_csv.append(values_NC)
                                      
x = "hands_total, noContact_total, noContact_left, noContact_left_upper, noContact_left_lower, noContact_left_ipsi, noContact_left_contra, noContact_left_ULquad, noContact_left_LLquad, noContact_left_URquad, noContact_left_LRquad, noContact_hands_right, noContact_right_upper, noContact_right_lower, noContact_right_contra, noContact_right_ipsi, noContact_right_ULquad, noContact_right_LLquad, noContact_right_URquad, noContact_right_LRquad"
noContact_csv.insert(0, x)

textfile_NC = open("leftHand_hands_noContact_analysis.csv", "w")
for element in noContact_csv:
    textfile_NC.write(element + "\n")
textfile_NC.close()

In [ ]:
leftH_noContact = pd.read_csv('/content/gdrive/MyDrive/HandVisualField_Paper/hands_big_Csv_4_analysis/leftHand_hands_noContact_analysis.csv')
leftH_noContact

,hands_total,noContact_total,noContact_left,noContact_left_upper,noContact_left_lower,noContact_left_ipsi,noContact_left_contra,noContact_left_ULquad,noContact_left_LLquad,noContact_left_URquad,noContact_left_LRquad,noContact_hands_right,noContact_right_upper,noContact_right_lower,noContact_right_contra,noContact_right_ipsi,noContact_right_ULquad,noContact_right_LLquad,noContact_right_URquad,noContact_right_LRquad
0,4333315,582366,262129,32216,229913,208686,53443,28112,180574,4104,49339,320237,58713,261524,34570,285667,7424,27146,51289,234378


In [ ]:
#for loop to pull data from all csv's and group info into the columns mentioned below - object interaction-Portable

import pandas as pd
df = ldf

hands_csv2 = []

vid_id = str(df['Participant'])
vid_name = vid_id[:-4]
hands_left = df[' Hand'] == 'left'
hands_right = df[' Hand'] == 'right'
hands_lower = df['cart_Centre_Y'] < 128
hands_upper = df['cart_Centre_Y'] >= 128
hands_Semi_Right = df[' Centre_X'] > 228
hands_Semi_Left = df[' Centre_X'] <= 228
hands_noContact = df[' Contact'] == 'NO_CONTACT'
hands_Portable = df[' Contact'] == 'PORTABLE_OBJECT'
hands_Stationary  = df[' Contact'] == 'STATIONARY_OBJECT'
hands_Self = df[' Contact'] == 'SELF_CONTACT'
  
values1 = str(len(df[' Hand'])) + "," + str(len(df[hands_Portable])) + "," + str(len(df[hands_Portable & hands_left])) + "," + str(len(df[hands_Portable & hands_left & hands_upper])) + "," + str(len(df[hands_Portable & hands_left & hands_lower])) + "," + str(len(df[hands_Portable & hands_left & hands_Semi_Left])) + "," + str(len(df[hands_Portable & hands_left & hands_Semi_Right])) + "," + str(len(df[hands_Portable & hands_left & hands_upper & hands_Semi_Left])) + "," + str(len(df[hands_Portable & hands_left & hands_lower & hands_Semi_Left])) + "," + str(len(df[hands_Portable & hands_left & hands_upper & hands_Semi_Right])) + "," + str(len(df[hands_Portable & hands_left & hands_lower & hands_Semi_Right])) + "," + str(len(df[hands_Portable & hands_right])) + "," + str(len(df[hands_Portable & hands_right & hands_upper])) + "," + str(len(df[hands_Portable & hands_right & hands_lower])) + "," + str(len(df[hands_Portable & hands_right & hands_Semi_Left])) + "," + str(len(df[hands_Portable & hands_right & hands_Semi_Right])) + "," + str(len(df[hands_Portable & hands_right & hands_upper & hands_Semi_Left])) + "," + str(len(df[hands_Portable & hands_right & hands_lower & hands_Semi_Left])) + "," + str(len(df[hands_Portable & hands_right & hands_upper & hands_Semi_Right])) + "," + str(len(df[hands_Portable & hands_right & hands_lower & hands_Semi_Right])) 
hands_csv2.append(values1)
                                      
x = "hands_total, portable_total, portable_left, portable_left_upper, portable_left_lower, portable_left_ipsi, portable_left_contra, portable_left_ULquad, portable_left_LLquad, portable_left_URquad, portable_left_LRquad, portable_hands_right, portablet_right_upper, portable_right_lower, portable_right_contra, portable_right_ipsi, portable_right_ULquad, portable_right_LLquad, portable_right_URquad, portable_right_LRquad"
hands_csv2.insert(0, x)

textfile = open("leftHanders_Portable.csv", "w")
for element in hands_csv2:
    textfile.write(element + "\n")
textfile.close()

In [ ]:
leftH_portable = pd.read_csv('/content/gdrive/MyDrive/HandVisualField_Paper/hands_big_Csv_4_analysis/leftHanders_Portable.csv')
leftH_portable

,hands_total,portable_total,portable_left,portable_left_upper,portable_left_lower,portable_left_ipsi,portable_left_contra,portable_left_ULquad,portable_left_LLquad,portable_left_URquad,portable_left_LRquad,portable_hands_right,portablet_right_upper,portable_right_lower,portable_right_contra,portable_right_ipsi,portable_right_ULquad,portable_right_LLquad,portable_right_URquad,portable_right_LRquad
0,4333315,3534648,1862797,447608,1415189,1587239,275558,414311,1172928,33297,242261,1671851,529277,1142574,143924,1527927,56713,87211,472564,1055363


In [ ]:
#for loop to pull data from all csv's and group info into the columns mentioned below - object interaction-stationary

import pandas as pd
df = ldf

hands_csv3 = []

vid_id = str(df['Participant'])
vid_name = vid_id[:-4]
hands_left = df[' Hand'] == 'left'
hands_right = df[' Hand'] == 'right'
hands_lower = df['cart_Centre_Y'] < 128
hands_upper = df['cart_Centre_Y'] >= 128
hands_Semi_Right = df[' Centre_X'] > 228
hands_Semi_Left = df[' Centre_X'] <= 228
hands_noContact = df[' Contact'] == 'NO_CONTACT'
hands_Portable = df[' Contact'] == 'PORTABLE_OBJECT'
hands_Stationary  = df[' Contact'] == 'STATIONARY_OBJECT'
hands_Self = df[' Contact'] == 'SELF_CONTACT'
  
values1 = str(len(df[' Hand'])) + "," + str(len(df[hands_Stationary])) + "," + str(len(df[hands_Stationary & hands_left])) + "," + str(len(df[hands_Stationary & hands_left & hands_upper])) + "," + str(len(df[hands_Stationary & hands_left & hands_lower])) + "," + str(len(df[hands_Stationary & hands_left & hands_Semi_Left])) + "," + str(len(df[hands_Stationary & hands_left & hands_Semi_Right])) + "," + str(len(df[hands_Stationary & hands_left & hands_upper & hands_Semi_Left])) + "," + str(len(df[hands_Stationary & hands_left & hands_lower & hands_Semi_Left])) + "," + str(len(df[hands_Stationary & hands_left & hands_upper & hands_Semi_Right])) + "," + str(len(df[hands_Stationary & hands_left & hands_lower & hands_Semi_Right])) + "," + str(len(df[hands_Stationary & hands_right])) + "," + str(len(df[hands_Stationary & hands_right & hands_upper])) + "," + str(len(df[hands_Stationary & hands_right & hands_lower])) + "," + str(len(df[hands_Stationary & hands_right & hands_Semi_Left])) + "," + str(len(df[hands_Stationary & hands_right & hands_Semi_Right])) + "," + str(len(df[hands_Stationary & hands_right & hands_upper & hands_Semi_Left])) + "," + str(len(df[hands_Stationary & hands_right & hands_lower & hands_Semi_Left])) + "," + str(len(df[hands_Stationary & hands_right & hands_upper & hands_Semi_Right])) + "," + str(len(df[hands_Stationary & hands_right & hands_lower & hands_Semi_Right])) 
hands_csv3.append(values1)
                                      
x = "hands_total, Stationary_total, Stationary_left, Stationary_left_upper, Stationary_left_lower, Stationary_left_ipsi, Stationary_left_contra, Stationary_left_ULquad, Stationary_left_LLquad, Stationary_left_URquad, Stationary_left_LRquad, Stationary_hands_right, Stationary_right_upper, Stationary_right_lower, Stationary_right_contra, Stationary_right_ipsi, Stationary_right_ULquad, Stationary_right_LLquad, Stationary_right_URquad, Stationary_right_LRquad"
hands_csv3.insert(0, x)

textfile = open("leftHander_Stationary.csv", "w")
for element in hands_csv3:
    textfile.write(element + "\n")
textfile.close()

In [ ]:
lH_stat = pd.read_csv('/content/gdrive/MyDrive/HandVisualField_Paper/hands_big_Csv_4_analysis/leftHander_Stationary.csv')
lH_stat

,hands_total,Stationary_total,Stationary_left,Stationary_left_upper,Stationary_left_lower,Stationary_left_ipsi,Stationary_left_contra,Stationary_left_ULquad,Stationary_left_LLquad,Stationary_left_URquad,Stationary_left_LRquad,Stationary_hands_right,Stationary_right_upper,Stationary_right_lower,Stationary_right_contra,Stationary_right_ipsi,Stationary_right_ULquad,Stationary_right_LLquad,Stationary_right_URquad,Stationary_right_LRquad
0,4333315,167242,66899,15095,51804,56078,10821,12716,43362,2379,8442,100343,22397,77946,7781,92562,2630,5151,19767,72795


In [ ]:
#for loop to pull data from all csv's and group info into the columns mentioned below - object interaction-selfContact

import pandas as pd
df = ldf

hands_csv4 = []

vid_id = str(df['Participant'])
vid_name = vid_id[:-4]
hands_left = df[' Hand'] == 'left'
hands_right = df[' Hand'] == 'right'
hands_lower = df['cart_Centre_Y'] < 128
hands_upper = df['cart_Centre_Y'] >= 128
hands_Semi_Right = df[' Centre_X'] > 228
hands_Semi_Left = df[' Centre_X'] <= 228
hands_noContact = df[' Contact'] == 'NO_CONTACT'
hands_Portable = df[' Contact'] == 'PORTABLE_OBJECT'
hands_Stationary  = df[' Contact'] == 'STATIONARY_OBJECT'
hands_Self = df[' Contact'] == 'SELF_CONTACT'
  
values1 = str(len(df[' Hand'])) + "," + str(len(df[hands_Self])) + "," + str(len(df[hands_Self & hands_left])) + "," + str(len(df[hands_Self & hands_left & hands_upper])) + "," + str(len(df[hands_Self & hands_left & hands_lower])) + "," + str(len(df[hands_Self & hands_left & hands_Semi_Left])) + "," + str(len(df[hands_Self & hands_left & hands_Semi_Right])) + "," + str(len(df[hands_Self & hands_left & hands_upper & hands_Semi_Left])) + "," + str(len(df[hands_Self & hands_left & hands_lower & hands_Semi_Left])) + "," + str(len(df[hands_Self & hands_left & hands_upper & hands_Semi_Right])) + "," + str(len(df[hands_Self & hands_left & hands_lower & hands_Semi_Right])) + "," + str(len(df[hands_Self & hands_right])) + "," + str(len(df[hands_Self & hands_right & hands_upper])) + "," + str(len(df[hands_Self & hands_right & hands_lower])) + "," + str(len(df[hands_Self & hands_right & hands_Semi_Left])) + "," + str(len(df[hands_Self & hands_right & hands_Semi_Right])) + "," + str(len(df[hands_Self & hands_right & hands_upper & hands_Semi_Left])) + "," + str(len(df[hands_Self & hands_right & hands_lower & hands_Semi_Left])) + "," + str(len(df[hands_Self & hands_right & hands_upper & hands_Semi_Right])) + "," + str(len(df[hands_Self & hands_right & hands_lower & hands_Semi_Right])) 
hands_csv4.append(values1)
                                      
x = "hands_total, hands_Self_total, hands_Self_left, hands_Self_left_upper, hands_Self_left_lower, hands_Self_left_ipsi, hands_Self_left_contra, hands_Self_left_ULquad, hands_Self_left_LLquad, hands_Self_left_URquad, hands_Self_left_LRquad, hands_Self_hands_right, hands_Self_right_upper, hands_Self_right_lower, hands_Self_right_contra, hands_Self_right_ipsi, hands_Self_right_ULquad, hands_Self_right_LLquad, hands_Self_right_URquad, hands_Self_right_LRquad"
hands_csv4.insert(0, x)

textfile = open("leftHanders_self.csv", "w")
for element in hands_csv4:
    textfile.write(element + "\n")
textfile.close()

In [ ]:
lH_self = pd.read_csv('/content/gdrive/MyDrive/HandVisualField_Paper/hands_big_Csv_4_analysis/leftHanders_self.csv')
lH_self

,hands_total,hands_Self_total,hands_Self_left,hands_Self_left_upper,hands_Self_left_lower,hands_Self_left_ipsi,hands_Self_left_contra,hands_Self_left_ULquad,hands_Self_left_LLquad,hands_Self_left_URquad,hands_Self_left_LRquad,hands_Self_hands_right,hands_Self_right_upper,hands_Self_right_lower,hands_Self_right_contra,hands_Self_right_ipsi,hands_Self_right_ULquad,hands_Self_right_LLquad,hands_Self_right_URquad,hands_Self_right_LRquad
0,4333315,49059,25330,4719,20611,15315,10015,3658,11657,1061,8954,23729,4656,19073,6875,16854,1703,5172,2953,13901


#Extracting data from dataset by hand dominance
## 7.3 - Right handed participants only

In [ ]:
#accessing the specific folder where the csv's are

import glob

csvs_folder_right = sorted(glob.glob(r'//content/gdrive/MyDrive/HandVisualField_Paper/hands_csvs_right_4_analysis/*.csv'))

print(len(csvs_folder_right))

565


In [ ]:
# Concatenate all data into one DataFrame

import pandas as pd

dfs_right = []

for filename in csvs_folder_right:

    dfs_right.append(pd.read_csv(filename))

big_frame_right = pd.concat(dfs_right, ignore_index=True)

big_frame_right

In [ ]:
#Turning Centre_Y column into a list
#Centre_Y was calculated based on image classification so 0 is the top left corner
#Centre_Y therefore needs to be converted to cartesian so 0 is the bottom left corner

Centre_Y_list = big_frame_right[" Centre_Y"].tolist()
Centre_Y_list[:10] #printing first 10 values to check they make sense

[147.5, 141.5, 126.0, 122.0, 118.5, 118.5, 118.0, 117.0, 123.0, 124.0]

In [ ]:
#Converting Y_centre to cartesian 
#Add values to new column
new_Centre_Y_list = []

for c in Centre_Y_list:
  n = 256 - c
  new_Centre_Y_list.append(n)

new_Centre_Y_list[:10]

[108.5, 114.5, 130.0, 134.0, 137.5, 137.5, 138.0, 139.0, 133.0, 132.0]

In [ ]:
#Adding new column to dataframe
big_frame_right.insert(loc = 3,
          column = 'cart_Centre_Y',
          value = new_Centre_Y_list)

# show the dataframe
big_frame_right

In [ ]:
#Removing Centre_Y column
big_frame_right.drop(big_frame_right.columns[4], axis=1, inplace=True)

In [ ]:
big_frame_right

,Participant,Frame,Hand,cart_Centre_Y,Centre_X,Contact
0,P01_01.pkl,211,right,108.5,321.0,STATIONARY_OBJECT
1,P01_01.pkl,212,right,114.5,324.5,STATIONARY_OBJECT
2,P01_01.pkl,213,right,130.0,328.5,PORTABLE_OBJECT
3,P01_01.pkl,214,right,134.0,329.0,PORTABLE_OBJECT
4,P01_01.pkl,215,right,137.5,326.5,PORTABLE_OBJECT
...,...,...,...,...,...,...
26900202,P37_103.pkl,19700,right,53.0,399.5,NO_CONTACT
26900203,P37_103.pkl,19701,right,60.5,406.0,NO_CONTACT
26900204,P37_103.pkl,19702,right,69.5,414.0,NO_CONTACT
26900205,P37_103.pkl,19703,right,85.5,421.0,NO_CONTACT


In [ ]:
#Saving the dataframe to a csv file
big_frame_right.to_csv(r'/content/gdrive/MyDrive/HandVisualField_Paper/hands_big_Csv_4_analysis/Final_Big_frame_Right.csv', index = False)

In [ ]:
%cd /content/gdrive/MyDrive/HandVisualField_Paper/hands_big_Csv_4_analysis

/content/gdrive/MyDrive/HandVisualField_Paper/hands_big_Csv_4_analysis


In [ ]:
import pandas as pd

df = pd.read_csv('/content/gdrive/MyDrive/HandVisualField_Paper/hands_big_Csv_4_analysis/Final_Big_frame_Right.csv')


In [ ]:
#for loop to pull data from the csv and group info into the columns mentioned below - hand position in space in right dominant participants

hands_csv = []

vid_id = str(df['Participant'])
vid_name = vid_id[:-4]
hands_left = df[' Hand'] == 'left'
hands_right = df[' Hand'] == 'right'
hands_lower = df['cart_Centre_Y'] < 128
hands_upper = df['cart_Centre_Y'] >= 128
hands_Semi_Right = df[' Centre_X'] > 228
hands_Semi_Left = df[' Centre_X'] <= 228

values = str(len(df[' Hand'])) + "," + str(len(df[hands_left])) + "," + str(len(df[hands_left & hands_upper])) + "," + str(len(df[hands_left & hands_lower])) + "," + str(len(df[hands_left & hands_Semi_Left])) + "," + str(len(df[hands_left & hands_Semi_Right])) + "," + str(len(df[hands_left & hands_upper & hands_Semi_Left])) + "," + str(len(df[hands_left & hands_lower & hands_Semi_Left])) + "," + str(len(df[hands_left & hands_upper & hands_Semi_Right])) + "," + str(len(df[hands_left & hands_lower & hands_Semi_Right])) + "," + str(len(df[hands_right])) + "," + str(len(df[hands_right & hands_upper])) + "," + str(len(df[hands_right & hands_lower]))+ ","  + str(len(df[hands_right & hands_Semi_Left])) + "," +str(len(df[hands_right & hands_Semi_Right])) + "," + str(len(df[hands_right & hands_upper & hands_Semi_Left])) + "," + str(len(df[hands_right & hands_lower & hands_Semi_Left])) + "," + str(len(df[hands_right & hands_upper & hands_Semi_Right])) + "," + str(len(df[hands_right & hands_lower & hands_Semi_Right])) 
  
hands_csv.append(values)
                                 
x = "hands_total, hands_left, left_upper, left_lower, left_ipsi, left_contra, left_ULquad, left_LLquad, leftURquad, leftLRquad, hands_right, right_upper, right_lower, right_contra, right_ipsi, right_ULquad, right_LLquad, right_URquad, right_LRquad" 
  
hands_csv.insert(0, x)

textfile = open("hands_dataset_analysis_Right.csv", "w")
for element in hands_csv:
    textfile.write(element + "\n")
textfile.close()

In [ ]:
crushed_df_Right = pd.read_csv('/content/gdrive/MyDrive/HandVisualField_Paper/hands_big_Csv_4_analysis/hands_dataset_analysis_Right.csv')
crushed_df_Right

,hands_total,hands_left,left_upper,left_lower,left_ipsi,left_contra,left_ULquad,left_LLquad,leftURquad,leftLRquad,hands_right,right_upper,right_lower,right_contra,right_ipsi,right_ULquad,right_LLquad,right_URquad,right_LRquad
0,26966886,12959122,2750823,10208299,10918940,2040182,2322655,8596285,428168,1612014,14007764,3662830,10344934,1294454,12713310,270375,1024079,3392455,9320855


In [ ]:
#for loop to pull data from all csv's and group info into the columns mentioned below - object interaction-noContact
df_NC = df

noContact_csv = []

vid_id = str(df_NC['Participant'])
vid_name = vid_id[:-4]
hands_left = df_NC[' Hand'] == 'left'
hands_right = df_NC[' Hand'] == 'right'
hands_lower = df_NC['cart_Centre_Y'] < 128
hands_upper = df_NC['cart_Centre_Y'] >= 128
hands_Semi_Right = df_NC[' Centre_X'] > 228
hands_Semi_Left = df_NC[' Centre_X'] <= 228
hands_noContact = df_NC[' Contact'] == 'NO_CONTACT'
hands_Portable = df_NC[' Contact'] == 'PORTABLE_OBJECT'
hands_Stationary  = df_NC[' Contact'] == 'STATIONARY_OBJECT'
hands_Self = df_NC[' Contact'] == 'SELF_CONTACT'
  
values_NC = str(len(df_NC[' Hand'])) + "," + str(len(df_NC[hands_noContact])) + "," + str(len(df_NC[hands_noContact & hands_left])) + "," + str(len(df_NC[hands_noContact & hands_left & hands_upper])) + "," + str(len(df_NC[hands_noContact & hands_left & hands_lower])) + "," + str(len(df_NC[hands_noContact & hands_left & hands_Semi_Left])) + "," + str(len(df_NC[hands_noContact & hands_left & hands_Semi_Right])) + "," + str(len(df_NC[hands_noContact & hands_left & hands_upper & hands_Semi_Left])) + "," + str(len(df_NC[hands_noContact & hands_left & hands_lower & hands_Semi_Left])) + "," + str(len(df_NC[hands_noContact & hands_left & hands_upper & hands_Semi_Right])) + "," + str(len(df_NC[hands_noContact & hands_left & hands_lower & hands_Semi_Right])) + "," + str(len(df_NC[hands_noContact & hands_right])) + "," + str(len(df_NC[hands_noContact & hands_right & hands_upper])) + "," + str(len(df_NC[hands_noContact & hands_right & hands_lower])) + "," + str(len(df_NC[hands_noContact & hands_right & hands_Semi_Left])) + "," + str(len(df_NC[hands_noContact & hands_right & hands_Semi_Right])) + "," + str(len(df_NC[hands_noContact & hands_right & hands_upper & hands_Semi_Left])) + "," + str(len(df_NC[hands_noContact & hands_right & hands_lower & hands_Semi_Left])) + "," + str(len(df_NC[hands_noContact & hands_right & hands_upper & hands_Semi_Right])) + "," + str(len(df_NC[hands_noContact & hands_right & hands_lower & hands_Semi_Right])) 
noContact_csv.append(values_NC)
                                      
x = "hands_total, noContact_total, noContact_left, noContact_left_upper, noContact_left_lower, noContact_left_ipsi, noContact_left_contra, noContact_left_ULquad, noContact_left_LLquad, noContact_left_URquad, noContact_left_LRquad, noContact_hands_right, noContact_right_upper, noContact_right_lower, noContact_right_contra, noContact_right_ipsi, noContact_right_ULquad, noContact_right_LLquad, noContact_right_URquad, noContact_right_LRquad"
noContact_csv.insert(0, x)

textfile_NC = open("rightHand_hands_noContact_analysis.csv", "w")
for element in noContact_csv:
    textfile_NC.write(element + "\n")
textfile_NC.close()

In [ ]:
rH_noContact = pd.read_csv('/content/gdrive/MyDrive/HandVisualField_Paper/hands_big_Csv_4_analysis/rightHand_hands_noContact_analysis.csv')
rH_noContact

,hands_total,noContact_total,noContact_left,noContact_left_upper,noContact_left_lower,noContact_left_ipsi,noContact_left_contra,noContact_left_ULquad,noContact_left_LLquad,noContact_left_URquad,noContact_left_LRquad,noContact_hands_right,noContact_right_upper,noContact_right_lower,noContact_right_contra,noContact_right_ipsi,noContact_right_ULquad,noContact_right_LLquad,noContact_right_URquad,noContact_right_LRquad
0,26966886,3386497,1829837,255624,1574213,1543051,286786,213416,1329635,42208,244578,1556660,329440,1227220,206935,1349725,37387,169548,292053,1057672


In [ ]:
#for loop to pull data from all csv's and group info into the columns mentioned below - object interaction-Portable

hands_csv2 = []

vid_id = str(df['Participant'])
vid_name = vid_id[:-4]
hands_left = df[' Hand'] == 'left'
hands_right = df[' Hand'] == 'right'
hands_lower = df['cart_Centre_Y'] < 128
hands_upper = df['cart_Centre_Y'] >= 128
hands_Semi_Right = df[' Centre_X'] > 228
hands_Semi_Left = df[' Centre_X'] <= 228
hands_noContact = df[' Contact'] == 'NO_CONTACT'
hands_Portable = df[' Contact'] == 'PORTABLE_OBJECT'
hands_Stationary  = df[' Contact'] == 'STATIONARY_OBJECT'
hands_Self = df[' Contact'] == 'SELF_CONTACT'
  
values1 = str(len(df[' Hand'])) + "," + str(len(df[hands_Portable])) + "," + str(len(df[hands_Portable & hands_left])) + "," + str(len(df[hands_Portable & hands_left & hands_upper])) + "," + str(len(df[hands_Portable & hands_left & hands_lower])) + "," + str(len(df[hands_Portable & hands_left & hands_Semi_Left])) + "," + str(len(df[hands_Portable & hands_left & hands_Semi_Right])) + "," + str(len(df[hands_Portable & hands_left & hands_upper & hands_Semi_Left])) + "," + str(len(df[hands_Portable & hands_left & hands_lower & hands_Semi_Left])) + "," + str(len(df[hands_Portable & hands_left & hands_upper & hands_Semi_Right])) + "," + str(len(df[hands_Portable & hands_left & hands_lower & hands_Semi_Right])) + "," + str(len(df[hands_Portable & hands_right])) + "," + str(len(df[hands_Portable & hands_right & hands_upper])) + "," + str(len(df[hands_Portable & hands_right & hands_lower])) + "," + str(len(df[hands_Portable & hands_right & hands_Semi_Left])) + "," + str(len(df[hands_Portable & hands_right & hands_Semi_Right])) + "," + str(len(df[hands_Portable & hands_right & hands_upper & hands_Semi_Left])) + "," + str(len(df[hands_Portable & hands_right & hands_lower & hands_Semi_Left])) + "," + str(len(df[hands_Portable & hands_right & hands_upper & hands_Semi_Right])) + "," + str(len(df[hands_Portable & hands_right & hands_lower & hands_Semi_Right])) 
hands_csv2.append(values1)
                                      
x = "hands_total, portable_total, portable_left, portable_left_upper, portable_left_lower, portable_left_ipsi, portable_left_contra, portable_left_ULquad, portable_left_LLquad, portable_left_URquad, portable_left_LRquad, portable_hands_right, portablet_right_upper, portable_right_lower, portable_right_contra, portable_right_ipsi, portable_right_ULquad, portable_right_LLquad, portable_right_URquad, portable_right_LRquad"
hands_csv2.insert(0, x)

textfile = open("rightHanders_Portable.csv", "w")
for element in hands_csv2:
    textfile.write(element + "\n")
textfile.close()

In [ ]:
rH_port = pd.read_csv('/content/gdrive/MyDrive/HandVisualField_Paper/hands_big_Csv_4_analysis/rightHanders_Portable.csv')
rH_port

,hands_total,portable_total,portable_left,portable_left_upper,portable_left_lower,portable_left_ipsi,portable_left_contra,portable_left_ULquad,portable_left_LLquad,portable_left_URquad,portable_left_LRquad,portable_hands_right,portablet_right_upper,portable_right_lower,portable_right_contra,portable_right_ipsi,portable_right_ULquad,portable_right_LLquad,portable_right_URquad,portable_right_LRquad
0,26966886,22520178,10600702,2370946,8229756,8964352,1636350,2007885,6956467,363061,1273289,11919476,3168938,8750538,999316,10920160,210747,788569,2958191,7961969


In [ ]:
#for loop to pull data from all csv's and group info into the columns mentioned below - object interaction-stationary

hands_csv3 = []

vid_id = str(df['Participant'])
vid_name = vid_id[:-4]
hands_left = df[' Hand'] == 'left'
hands_right = df[' Hand'] == 'right'
hands_lower = df['cart_Centre_Y'] < 128
hands_upper = df['cart_Centre_Y'] >= 128
hands_Semi_Right = df[' Centre_X'] > 228
hands_Semi_Left = df[' Centre_X'] <= 228
hands_noContact = df[' Contact'] == 'NO_CONTACT'
hands_Portable = df[' Contact'] == 'PORTABLE_OBJECT'
hands_Stationary  = df[' Contact'] == 'STATIONARY_OBJECT'
hands_Self = df[' Contact'] == 'SELF_CONTACT'
  
values1 = str(len(df[' Hand'])) + "," + str(len(df[hands_Stationary])) + "," + str(len(df[hands_Stationary & hands_left])) + "," + str(len(df[hands_Stationary & hands_left & hands_upper])) + "," + str(len(df[hands_Stationary & hands_left & hands_lower])) + "," + str(len(df[hands_Stationary & hands_left & hands_Semi_Left])) + "," + str(len(df[hands_Stationary & hands_left & hands_Semi_Right])) + "," + str(len(df[hands_Stationary & hands_left & hands_upper & hands_Semi_Left])) + "," + str(len(df[hands_Stationary & hands_left & hands_lower & hands_Semi_Left])) + "," + str(len(df[hands_Stationary & hands_left & hands_upper & hands_Semi_Right])) + "," + str(len(df[hands_Stationary & hands_left & hands_lower & hands_Semi_Right])) + "," + str(len(df[hands_Stationary & hands_right])) + "," + str(len(df[hands_Stationary & hands_right & hands_upper])) + "," + str(len(df[hands_Stationary & hands_right & hands_lower])) + "," + str(len(df[hands_Stationary & hands_right & hands_Semi_Left])) + "," + str(len(df[hands_Stationary & hands_right & hands_Semi_Right])) + "," + str(len(df[hands_Stationary & hands_right & hands_upper & hands_Semi_Left])) + "," + str(len(df[hands_Stationary & hands_right & hands_lower & hands_Semi_Left])) + "," + str(len(df[hands_Stationary & hands_right & hands_upper & hands_Semi_Right])) + "," + str(len(df[hands_Stationary & hands_right & hands_lower & hands_Semi_Right])) 
hands_csv3.append(values1)
                                      
x = "hands_total, Stationary_total, Stationary_left, Stationary_left_upper, Stationary_left_lower, Stationary_left_ipsi, Stationary_left_contra, Stationary_left_ULquad, Stationary_left_LLquad, Stationary_left_URquad, Stationary_left_LRquad, Stationary_hands_right, Stationary_right_upper, Stationary_right_lower, Stationary_right_contra, Stationary_right_ipsi, Stationary_right_ULquad, Stationary_right_LLquad, Stationary_right_URquad, Stationary_right_LRquad"
hands_csv3.insert(0, x)

textfile = open("rightHander_Stationary.csv", "w")
for element in hands_csv3:
    textfile.write(element + "\n")
textfile.close()

In [ ]:
rH_Stat = pd.read_csv('/content/gdrive/MyDrive/HandVisualField_Paper/hands_big_Csv_4_analysis/rightHander_Stationary.csv')
rH_Stat

,hands_total,Stationary_total,Stationary_left,Stationary_left_upper,Stationary_left_lower,Stationary_left_ipsi,Stationary_left_contra,Stationary_left_ULquad,Stationary_left_LLquad,Stationary_left_URquad,Stationary_left_LRquad,Stationary_hands_right,Stationary_right_upper,Stationary_right_lower,Stationary_right_contra,Stationary_right_ipsi,Stationary_right_ULquad,Stationary_right_LLquad,Stationary_right_URquad,Stationary_right_LRquad
0,26966886,734673,360784,91293,269491,312771,48013,80091,232680,11202,36811,373889,133673,240216,44744,329145,13654,31090,120019,209126


In [ ]:
#for loop to pull data from all csv's and group info into the columns mentioned below - object interaction-selfContact

hands_csv4 = []

vid_id = str(df['Participant'])
vid_name = vid_id[:-4]
hands_left = df[' Hand'] == 'left'
hands_right = df[' Hand'] == 'right'
hands_lower = df['cart_Centre_Y'] < 128
hands_upper = df['cart_Centre_Y'] >= 128
hands_Semi_Right = df[' Centre_X'] > 228
hands_Semi_Left = df[' Centre_X'] <= 228
hands_noContact = df[' Contact'] == 'NO_CONTACT'
hands_Portable = df[' Contact'] == 'PORTABLE_OBJECT'
hands_Stationary  = df[' Contact'] == 'STATIONARY_OBJECT'
hands_Self = df[' Contact'] == 'SELF_CONTACT'
  
values1 = str(len(df[' Hand'])) + "," + str(len(df[hands_Self])) + "," + str(len(df[hands_Self & hands_left])) + "," + str(len(df[hands_Self & hands_left & hands_upper])) + "," + str(len(df[hands_Self & hands_left & hands_lower])) + "," + str(len(df[hands_Self & hands_left & hands_Semi_Left])) + "," + str(len(df[hands_Self & hands_left & hands_Semi_Right])) + "," + str(len(df[hands_Self & hands_left & hands_upper & hands_Semi_Left])) + "," + str(len(df[hands_Self & hands_left & hands_lower & hands_Semi_Left])) + "," + str(len(df[hands_Self & hands_left & hands_upper & hands_Semi_Right])) + "," + str(len(df[hands_Self & hands_left & hands_lower & hands_Semi_Right])) + "," + str(len(df[hands_Self & hands_right])) + "," + str(len(df[hands_Self & hands_right & hands_upper])) + "," + str(len(df[hands_Self & hands_right & hands_lower])) + "," + str(len(df[hands_Self & hands_right & hands_Semi_Left])) + "," + str(len(df[hands_Self & hands_right & hands_Semi_Right])) + "," + str(len(df[hands_Self & hands_right & hands_upper & hands_Semi_Left])) + "," + str(len(df[hands_Self & hands_right & hands_lower & hands_Semi_Left])) + "," + str(len(df[hands_Self & hands_right & hands_upper & hands_Semi_Right])) + "," + str(len(df[hands_Self & hands_right & hands_lower & hands_Semi_Right])) 
hands_csv4.append(values1)
                                      
x = "hands_total, hands_Self_total, hands_Self_left, hands_Self_left_upper, hands_Self_left_lower, hands_Self_left_ipsi, hands_Self_left_contra, hands_Self_left_ULquad, hands_Self_left_LLquad, hands_Self_left_URquad, hands_Self_left_LRquad, hands_Self_hands_right, hands_Self_right_upper, hands_Self_right_lower, hands_Self_right_contra, hands_Self_right_ipsi, hands_Self_right_ULquad, hands_Self_right_LLquad, hands_Self_right_URquad, hands_Self_right_LRquad"
hands_csv4.insert(0, x)

textfile = open("rightHanders_self.csv", "w")
for element in hands_csv4:
    textfile.write(element + "\n")
textfile.close()

In [ ]:
rH_self = pd.read_csv('/content/gdrive/MyDrive/HandVisualField_Paper/hands_big_Csv_4_analysis/rightHanders_self.csv')
rH_self

,hands_total,hands_Self_total,hands_Self_left,hands_Self_left_upper,hands_Self_left_lower,hands_Self_left_ipsi,hands_Self_left_contra,hands_Self_left_ULquad,hands_Self_left_LLquad,hands_Self_left_URquad,hands_Self_left_LRquad,hands_Self_hands_right,hands_Self_right_upper,hands_Self_right_lower,hands_Self_right_contra,hands_Self_right_ipsi,hands_Self_right_ULquad,hands_Self_right_LLquad,hands_Self_right_URquad,hands_Self_right_LRquad
0,26966886,325538,167799,32960,134839,98766,69033,21263,77503,11697,57336,157739,30779,126960,43459,114280,8587,34872,22192,92088


In [ ]:
#Checking the overall number of frames is equal to the sum of right and left dominant frames
nLeft = 4333315
nRight = 26966886
nTotal = 31300201

nLeft + nRight == nTotal

True

In [ ]:
# Install libraries to create the PDF
%%capture
!apt update
!apt install texlive-xetex texlive-fonts-recommended texlive-generic-recommended

In [ ]:
%cd #your destiny folder - remover this text in green once you have added the directory

/content/gdrive/MyDrive/HandVisualField_Paper


In [ ]:
#Exports the jupyter notebook to pdf
!jupyter nbconvert VisualField_Methods0.ipynb --to pdf --output ./VisualField_Methods0.pdf

[NbConvertApp] Converting notebook VisualField_Methods0.ipynb to pdf
[NbConvertApp] Writing 244143 bytes to ./notebook.tex
[NbConvertApp] Building PDF
[NbConvertApp] Running xelatex 3 times: ['xelatex', './notebook.tex', '-quiet']
[NbConvertApp] Running bibtex 1 time: ['bibtex', './notebook']
[NbConvertApp] WARNING | bibtex had problems, most likely because there were no citations
[NbConvertApp] PDF successfully created
[NbConvertApp] Writing 168109 bytes to ./VisualField_Methods0.pdf
